In [1]:
from networks.attention_agent import CausalAttentionAgent
import torch
from utils.trading_gym_env import TradingEnv
from utils.synthetic_data_service import SyntheticOHLCVGenerator
import pandas as pd
import matplotlib.pyplot as plt
from diffevo import DDIMScheduler, BayesianGenerator
from torch.nn.utils import parameters_to_vector, vector_to_parameters
from utils.fitess_funcs import batched_fitness_function
from api_wrappers.kraken_wrapper import KrakenWrapper
kw = KrakenWrapper()
dfs = kw.load_hist_files()

c:\Users\nick5\miniconda3\envs\tree-env\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
import numpy as np
import random

print(len(dfs))

def add_features(df):
    df.columns = [col.lower() for col in df.columns]
    print(df.head())
    # Ensure columns are numeric
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    df['high'] = pd.to_numeric(df['high'], errors='coerce')
    df['low'] = pd.to_numeric(df['low'], errors='coerce')
    df['vol'] = pd.to_numeric(df['vol'], errors='coerce')

    # Add log returns
    df['log_return'] = np.log(df['close']).diff()

    # Add moving averages
    df['ma_10'] = df['close'].rolling(window=10).mean()
    df['ma_50'] = df['close'].rolling(window=50).mean()

    # Add Fibonacci levels
    fib_ratios = [0.0, 0.236, 0.382, 0.5, 0.618, 1.0]
    for ratio in fib_ratios:
        df[f'fib_{int(ratio * 1000)}'] = np.nan
    for i in range(20, len(df)):
        high = df['high'].iloc[i-20:i].max()
        low = df['low'].iloc[i-20:i].min()
        for ratio in fib_ratios:
            level = high - (high - low) * ratio
            df.at[i, f'fib_{int(ratio * 1000)}'] = level

    # Drop NaN rows
    df = df.dropna().reset_index(drop=True)
    return df
train_size = int(len(list(dfs.items())[0][1]) * 0.6)
# Add features to all dataframes
dfs = {asset: add_features(df) for asset, df in dfs.items()}
def split_train_test(df, train_ratio=0.8):
    train_df = df.iloc[:train_size]
    test_df = df.iloc[train_size:]
    return train_df, test_df

# Split all dataframes into train/test sets
train_test_data = {asset: split_train_test(df) for asset, df in dfs.items()}
def prepare_data(df, seq_len):
    states = []
    price_changes = []

    for i in range(len(df) - seq_len):
        state = df.iloc[i:i+seq_len][['log_return', 'ma_10', 'ma_50']].values
        price_change = df.iloc[i+seq_len]['log_return']
        states.append(state)
        price_changes.append(price_change)

    states = torch.tensor(states, dtype=torch.float32)  # Shape: (num_samples, seq_len, num_features)
    price_changes = torch.tensor(price_changes, dtype=torch.float32)  # Shape: (num_samples,)
    return states, price_changes

# Prepare data for all assets
seq_len = 144
train_data = {asset: prepare_data(train, seq_len) for asset, (train, _) in train_test_data.items()}
test_data = {asset: prepare_data(test, seq_len) for asset, (_, test) in train_test_data.items()}

22
         date      open      high       low     close      vwap           vol
0  1694390400  0.249169  0.249169  0.236997  0.242000  0.241745  1.630805e+06
1  1694476800  0.242000  0.252356  0.240993  0.245998  0.248591  1.011622e+06
2  1694563200  0.245534  0.249705  0.243873  0.248728  0.247799  2.463036e+05
3  1694649600  0.248832  0.252286  0.246532  0.251329  0.249129  1.596386e+05
4  1694736000  0.251267  0.254644  0.237800  0.250528  0.244553  3.753199e+06
         date     open     high      low    close     vwap            vol
0  1694390400  0.09131  0.09202  0.08760  0.08844  0.08886   44115.023285
1  1694476800  0.08896  0.09198  0.08859  0.08965  0.09029  191172.611332
2  1694563200  0.08921  0.09195  0.08900  0.09089  0.09070   78639.702407
3  1694649600  0.09122  0.09379  0.09050  0.09347  0.09216  121614.765609
4  1694736000  0.09352  0.09671  0.09328  0.09630  0.09462   86333.150052
         date    open    high     low   close    vwap          vol
0  1694390400  1.2

C:\Users\nick5\AppData\Local\Temp\ipykernel_23640\6427933.py:56: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:281.)
  states = torch.tensor(states, dtype=torch.float32)  # Shape: (num_samples, seq_len, num_features)


In [ ]:
import torch.nn.functional as F

def run(x_array, population, agent):
    rewards = []
    # Example of a random walk in the environment
    for xp in population:
        vector_to_parameters(torch.tensor(xp, dtype=torch.float32), agent.parameters())
        rewards.append(batched_fitness_function(agent, x_array))
    return rewards

# Define hyperparameters
POP_SIZE = 42
SCALING = 0.1

# Instantiate the model
STATE_DIM = train_data[list(train_data.keys())[0]][0].shape[-1]  # Number of features
ACTION_DIM = 2  # Buy, Sell
EP_LEN = 55
agent_model = CausalAttentionAgent(state_dim=STATE_DIM, num_heads=8, embed_dim=432, action_dim=ACTION_DIM, seq_len=seq_len)

# Initialize population
dim = parameters_to_vector(agent_model.parameters()).shape[0]
population = torch.randn(POP_SIZE, dim) * SCALING

def fitness_function(population, agent, train_data, num_assets=10, device="cuda"):
    """
    Evaluate the fitness of the population on a randomly selected batch of assets.

    Args:
        population: The population of agent parameters.
        agent: The trading agent model.
        train_data: Dictionary of training data for all assets.
        num_assets: Number of assets to randomly select for evaluation.
        device: The device to run the computation on ("cuda" or "cpu").

    Returns:
        Tensor of normalized rewards for each member of the population.
    """
    # Move agent to the device
    agent.to(device)

    # Randomly select a batch of assets
    selected_assets = random.sample(list(train_data.keys()), num_assets)
    print(f"Selected assets for this generation: {selected_assets}")

    # Combine states and price changes for the selected assets
    batch_states = torch.stack([train_data[asset][0] for asset in selected_assets], dim=0).to(device)  # Shape: (batch, ep_len, seq_len, features)
    batch_price_changes = torch.stack([train_data[asset][1] for asset in selected_assets], dim=0).to(device)  # Shape: (batch, ep_len)

    # Initialize rewards for the population
    population_rewards = torch.zeros(len(population), len(selected_assets), device=device)  # Shape: (population_size, num_assets)

    for i, params in enumerate(population):
        # Update the agent's parameters
        vector_to_parameters(params.to(device), agent.parameters())

        # Evaluate the agent on the batch
        rewards = batched_fitness_function(agent, batch_states, batch_price_changes)  # Shape: (batch, 1)
        population_rewards[i] = rewards.squeeze()  # Store rewards for this parameter set

    # Normalize rewards per parameter set using softmax across assets
    normalized_rewards = F.softmax(population_rewards, dim=0)  # Normalize across assets for each parameter set

    # Sum normalized rewards across all assets for each parameter set
    final_rewards = normalized_rewards.sum(dim=1)  # Shape: (population_size,)

    return final_rewards

# Train with diffusion evolution
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

scheduler = DDIMScheduler(num_step=233)
population = population.to(device)  # Move population to GPU

for t, alpha in scheduler:  # Number of training steps
    rewards = fitness_function(population, agent_model, train_data, device=device)
    print(f"Step {t}, Best Reward: {max(rewards)}")
    population = population.cpu()  # Move population to CPU for generator
    generator = BayesianGenerator(population, rewards.cpu(), alpha)
    population = generator(noise=0).to(device)  # put new population on gpu

    best_para = population[-1]
    torch.save(best_para, 'best_attention_agent.pth')

Using device: cuda
Selected assets for this generation: ['EOSUSDT.txt', 'XTZUSDT.txt', 'XBTUSDT.txt', 'USDCUSDT.txt', 'BCHUSDT.txt', 'DOTUSDT.txt', 'USTUSDT.txt', 'LTCUSDT.txt', 'APEUSDT.txt', 'LINKUSDT.txt']
Step 232, Best Reward: 0.26497510075569153
Selected assets for this generation: ['BCHUSDT.txt', 'SHIBUSDT.txt', 'XBTUSDT.txt', 'DAIUSDT.txt', 'EOSUSDT.txt', 'APEUSDT.txt', 'LINKUSDT.txt', 'XDGUSDT.txt', 'XMRUSDT.txt', 'USTUSDT.txt']
Step 231, Best Reward: 0.26956456899642944
Selected assets for this generation: ['SHIBUSDT.txt', 'ETHUSDT.txt', 'APEUSDT.txt', 'ALGOUSDT.txt', 'DAIUSDT.txt', 'LTCUSDT.txt', 'USDCUSDT.txt', 'ADAUSDT.txt', 'XMRUSDT.txt', 'MANAUSDT.txt']
Step 230, Best Reward: 0.2962484657764435
Selected assets for this generation: ['MANAUSDT.txt', 'AVAXUSDT.txt', 'SOLUSDT.txt', 'EOSUSDT.txt', 'ALGOUSDT.txt', 'XRPUSDT.txt', 'XTZUSDT.txt', 'DOTUSDT.txt', 'USDCUSDT.txt', 'BCHUSDT.txt']
Step 229, Best Reward: 0.30556756258010864
Selected assets for this generation: ['ALGOUSD

In [ ]:
import matplotlib.pyplot as plt
# Load the saved population
saved_population_path = 'best_attention_agent.pth'
best_params = torch.load(saved_population_path)
agent_model = CausalAttentionAgent(state_dim=STATE_DIM, num_heads=8, embed_dim=432, action_dim=ACTION_DIM, seq_len=seq_len)
current_params = parameters_to_vector(agent_model.parameters())
print(current_params.shape)
print(best_params.shape)
vector_to_parameters(best_params, agent_model.parameters())
# Evaluate the best parameters on the test set and plot performance
def evaluate_population_on_test(agent, test_data, num_assets=10):
    """
    Evaluate the saved population on the test set and plot performance.

    Args:
        agent: The trading agent model.
        test_data: Dictionary of test data for all assets.
        num_assets: Number of assets to randomly select for evaluation.

    Returns:
        List of rewards for the best parameters on the test set.
    """
    # Randomly select a batch of assets
    selected_assets = random.sample(list(test_data.keys()), num_assets)
    print(f"Selected assets for evaluation: {selected_assets}")

    # Combine states and price changes for the selected assets
    batch_states = torch.stack([test_data[asset][0] for asset in selected_assets], dim=0)  # Shape: (batch, ep_len, seq_len, features)
    batch_price_changes = torch.stack([test_data[asset][1] for asset in selected_assets], dim=0)  # Shape: (batch, ep_len)

    # Load the best parameters into the agent
    vector_to_parameters(best_params.cpu(), agent.parameters())

    # Evaluate the agent on the test set
    batch, ep_len = batch_price_changes.shape
    print(batch_price_changes.shape)
    rewards = torch.zeros(batch, ep_len)  # Initialize rewards for each asset over time

    for t in range(ep_len - 1):
        # Get the current state for all assets at time t
        current_states = batch_states[:, t, :, :]  # Shape: (batch, seq_len, features)

        # Get actions for all assets in the batch
        actions = agent(current_states)  # Shape: (batch, 2) -> 2 actions: Buy, Sell
        actions = torch.argmax(actions, dim=-1)  # Convert to discrete actions (0=Sell, 1=Buy)

        # Convert actions to -1 (Sell) and 1 (Buy)
        #actions = actions.float() * 2 - 1  # Map (0, 1) -> (-1, 1)
        print(actions)
        # Calculate rewards as the product of actions and percentage changes
        rewards[:, t + 1] = rewards[:, t] + (actions * batch_price_changes[:, t + 1])

    # Plot cumulative rewards for each asset
    plt.figure(figsize=(12, 6))
    for i, asset in enumerate(selected_assets):
        plt.plot(rewards[i].numpy(), label=asset)
    plt.xlabel('Time Step')
    plt.ylabel('Cumulative Reward')
    plt.title('Agent Performance on Selected Test Assets')
    plt.legend()
    plt.show()

    return rewards

# Evaluate the best agent on the test set and plot performance
test_rewards = evaluate_population_on_test(agent_model, test_data, num_assets=10)
print("Test rewards for the best agent:", test_rewards)

C:\Users\nick5\AppData\Local\Temp\ipykernel_23640\3515018346.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_params = torch.load(saved_population_path)


NameError: name 'agent_model' is not defined